In [1]:
# !pip install langchain

In [2]:
# !pip install langchain-ollama langchain-chroma

In [3]:
# !pip install langchain_community chromadb

In [4]:
# !pip install -U langchain-ollama

In [1]:
from langchain_community.document_loaders import TextLoader
from concurrent.futures import ThreadPoolExecutor
from langchain.text_splitter import CharacterTextSplitter
from langchain_chroma import Chroma
import pandas as pd
from langchain_ollama import OllamaEmbeddings
# from chromadb.config import Settings
# from chromadb import Client
# from langchain.vectorstores import Chroma

In [2]:
books = pd.read_csv("./books_with_categories.csv")

In [3]:
books["tagged_description"].to_csv("tagged_description.txt", sep="\n", index=False, header=False)

In [4]:
tagged_description = TextLoader("tagged_description.txt", encoding="utf-8").load()
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
document = text_splitter.split_documents(tagged_description) 

Created a chunk of size 1168, which is longer than the specified 0
Created a chunk of size 1214, which is longer than the specified 0
Created a chunk of size 373, which is longer than the specified 0
Created a chunk of size 309, which is longer than the specified 0
Created a chunk of size 483, which is longer than the specified 0
Created a chunk of size 482, which is longer than the specified 0
Created a chunk of size 960, which is longer than the specified 0
Created a chunk of size 188, which is longer than the specified 0
Created a chunk of size 843, which is longer than the specified 0
Created a chunk of size 296, which is longer than the specified 0
Created a chunk of size 197, which is longer than the specified 0
Created a chunk of size 881, which is longer than the specified 0
Created a chunk of size 1088, which is longer than the specified 0
Created a chunk of size 1189, which is longer than the specified 0
Created a chunk of size 304, which is longer than the specified 0
Create

In [5]:
for idx, doc in enumerate(document):
    doc.metadata["categories"] = books["Maped_categories"][idx]
    print(doc.metadata)

{'source': 'tagged_description.txt', 'categories': 'Fiction'}
{'source': 'tagged_description.txt', 'categories': 'Fiction'}
{'source': 'tagged_description.txt', 'categories': 'Fiction'}
{'source': 'tagged_description.txt', 'categories': 'Nonfiction'}
{'source': 'tagged_description.txt', 'categories': 'Nonfiction'}
{'source': 'tagged_description.txt', 'categories': 'Nonfiction'}
{'source': 'tagged_description.txt', 'categories': 'Fiction'}
{'source': 'tagged_description.txt', 'categories': 'Fiction'}
{'source': 'tagged_description.txt', 'categories': 'Fiction'}
{'source': 'tagged_description.txt', 'categories': 'Fiction'}
{'source': 'tagged_description.txt', 'categories': 'Fiction'}
{'source': 'tagged_description.txt', 'categories': 'Fiction'}
{'source': 'tagged_description.txt', 'categories': 'Fiction'}
{'source': 'tagged_description.txt', 'categories': 'Fiction'}
{'source': 'tagged_description.txt', 'categories': 'Fiction'}
{'source': 'tagged_description.txt', 'categories': 'Fiction'}

In [ ]:
# Initialize Chroma and store embeddings
chroma_db = Chroma.from_documents(
    documents=document,
    embedding=OllamaEmbeddings(model="deepseek-r1"),
    collection_name="book_recommendation",  # Name your collection
    persist_directory="./chroma_db" 
)

In [78]:
def retrieve_similar_data(query, top_k=10):
    res = chroma_db.similarity_search(query, k = 50)
    res_books = []
    for i in range(len(res)):
        res_books += [int(res[i].page_content.strip('"').split()[0])]

    return books[books['isbn13'].isin(res_books)].head(top_k)

In [48]:
# chroma_db = Chroma(
#     persist_directory="./chroma_db",  # Same directory where you saved it
#     embedding_function=OllamaEmbeddings(model="deepseek-r1")  # Must match original function
# )

In [59]:
retrieve_similar_data("Books about crime and investigation with suspenseful plots")

,isbn13,isbn10,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
81,9780024181909,0024181900,William Strunk;Elwyn Brooks White,Language Arts & Disciplines,http://books.google.com/books/content?id=ADaMf...,The only style manual ever to appear on the be...,1979.0,4.20,85.0,204.0,The Elements of Style,9780024181909 The only style manual ever to ap...
279,9780060899226,0060899220,Anthony Bourdain,Biography & Autobiography,http://books.google.com/books/content?id=bxLbZ...,"A deliciously funny, delectably shocking banqu...",2007.0,4.06,312.0,168706.0,Kitchen Confidential Updated Ed: Adventures in...,"9780060899226 A deliciously funny, delectably ..."
570,9780140143454,0140143459,Michael M. Lewis,Biography & Autobiography,http://books.google.com/books/content?id=eboqx...,The author recounts his experiences on the luc...,1990.0,4.15,256.0,22710.0,Liar's Poker: Rising Through the Wreckage on W...,9780140143454 The author recounts his experien...
599,9780140238280,014023828X,T. Coraghessan Boyle,Fiction,http://books.google.com/books/content?id=ZA_BO...,The lives of two different couples--wealthy Lo...,1996.0,3.65,355.0,23410.0,The Tortilla Curtain,9780140238280 The lives of two different coupl...
919,9780156212502,0156212501,Virginia Woolf,Fiction,http://books.google.com/books/content?id=hZyix...,Collects nearly fifty short stories and sketch...,1989.0,4.04,345.0,684.0,The Complete Shorter Fiction of Virginia Woolf,9780156212502 Collects nearly fifty short stor...
923,9780156260374,0156260379,Virginia Woolf,Biography & Autobiography,http://books.google.com/books/content?id=FjEvj...,Entries interrupted only by her periodic break...,1980.0,4.37,384.0,374.0,The Diary of Virginia Woolf: 1920-1924,9780156260374 Entries interrupted only by her ...
925,9780156260398,0156260395,Virginia Woolf,Biography & Autobiography,http://books.google.com/books/content?id=ai7uA...,Entries interrupted only by her periodic break...,1983.0,4.44,420.0,213.0,The Diary of Virginia Woolf: 1931-1935,9780156260398 Entries interrupted only by her ...
927,9780156260558,0156260557,Italo Calvino,Fiction,http://books.google.com/books/content?id=6Ynoy...,In a collection of stories written during the ...,1984.0,3.93,300.0,3874.0,Difficult Loves,9780156260558 In a collection of stories writt...
930,9780156306300,0156306301,Stanislaw Lem,Fiction,http://books.google.com/books/content?id=4IZ3P...,When a crew of earthmen--among them a space pi...,1988.0,4.12,322.0,2125.0,Fiasco,9780156306300 When a crew of earthmen--among t...
931,9780156453806,0156453800,Italo Calvino,Fiction,http://books.google.com/books/content?id=5AokC...,"In Kublai Khan's garden, at sunset, the young ...",1978.0,4.17,165.0,39058.0,Invisible Cities,"9780156453806 In Kublai Khan's garden, at suns..."


In [61]:
from chromadb import PersistentClient

# Load the persisted database
client = PersistentClient(path="./chroma_db")
collection = client.get_collection("book_recommendation")


In [66]:
collection.peek(5)["metadatas"]

[{'source': 'tagged_description.txt'},
 {'source': 'tagged_description.txt'},
 {'source': 'tagged_description.txt'},
 {'source': 'tagged_description.txt'},
 {'source': 'tagged_description.txt'}]